In [1]:
import pandas as pd
import numpy as np

/home/fejiang/.local/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/fejiang/.local/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/tmp/ipykernel_241/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('worldcities.csv')
data.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6850,139.7514,Japan,JP,JPN,Tōkyō,primary,35676000.0,1392685764
1,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,19354922.0,1840034016
2,Mexico City,Mexico City,19.4424,-99.1310,Mexico,MX,MEX,Ciudad de México,primary,19028000.0,1484247881
3,Mumbai,Mumbai,19.0170,72.8570,India,IN,IND,Mahārāshtra,admin,18978000.0,1356226629
4,São Paulo,Sao Paulo,-23.5587,-46.6250,Brazil,BR,BRA,São Paulo,admin,18845000.0,1076532519


## EDA

In [3]:
# Cities in USA
only_us = data[data['country'] == 'United States']
print(f'Total number of US cities in dataset: {only_us.shape[0]}')
#print(f'Of these, the number of distinct cities names is: {}')

Total number of US cities in dataset: 7328


In [4]:
# Cities in other countries (non-USA)
other_country = data[data['country'] != 'United States']
print(f'Total number of foreign cities in dataset: {other_country.shape}')

Total number of foreign cities in dataset: (8165, 11)


In [5]:
# Total number of countries the dataset has record for, number of cities in the dataset
print(f'Total number of countries the dataset has record for: {data.country.nunique()}')
print(f'Number of cities in the dataset: {data.shape[0]}')

Total number of countries the dataset has record for: 223
Number of cities in the dataset: 15493


In [6]:
# Number of overlap
us_cities = only_us.city_ascii.value_counts()
num_distinct_us = len(us_cities)
num_us = only_us.shape[0]
overlap_distinct = [c for c in other_country.city_ascii.unique() if c in us_cities]
num_overlap = sum([us_cities[c] for c in overlap_distinct])
print(f'There are {num_overlap} cities in US that has same name as some foreign cities, with {len(overlap_distinct)} distinct city names.')

other_cities = other_country.city_ascii.value_counts()
num_overlap_foreign = sum([other_cities[c] for c in overlap_distinct])
country_overlap = other_country['country'][other_country.city_ascii.isin(overlap_distinct)]
print(f'There are {num_overlap_foreign} cities in other countries that has same name as some US cities, across {country_overlap.nunique()} different countries.')

There are 479 cities in US that has same name as some foreign cities, with 230 distinct city names.
There are 300 cities in other countries that has same name as some US cities, across 64 different countries.


In [7]:
country_overlap.unique()

array(['India', 'Egypt', 'Russia', 'France', 'United Kingdom', 'Peru',
       'Colombia', 'Australia', 'Germany', 'Italy', 'Greece', 'Portugal',
       'Syria', 'Austria', 'Canada', 'Bolivia', 'Poland', 'Uruguay',
       'Saudi Arabia', 'Costa Rica', 'Panama', 'Switzerland', 'Serbia',
       'Ireland', 'Liberia', 'Netherlands', 'Jamaica', 'Sierra Leone',
       'Latvia', 'New Zealand', 'Guyana', 'Barbados', 'Seychelles',
       'San Marino', 'Sint Maarten', 'Chile', 'Argentina', 'Iran',
       'Paraguay', 'Mexico', 'Puerto Rico', 'Cuba', 'Brazil',
       'Philippines', 'South Africa', 'Spain', 'El Salvador',
       'Trinidad And Tobago', 'Israel', 'Guatemala', 'Venezuela',
       'Dominican Republic', 'Romania', 'Namibia', 'Honduras',
       'Nicaragua', 'Belize', 'Malta', 'Micronesia, Federated States Of',
       'Sao Tome And Principe',
       'Saint Helena, Ascension, And Tristan Da Cunha', 'Bermuda',
       'Ecuador', 'Isle Of Man'], dtype=object)

## Data Processing

In [8]:
only_overlap_us = only_us[only_us.city_ascii.isin(overlap_distinct)]
only_overlap_other = other_country[other_country.city_ascii.isin(overlap_distinct)]

# This dataframe keeps the record in the form (1 US city to 1 Foreign city with same name)
# So for the same city in US, it might have multiple rows (1 row for each foreign city it has the same name with)
only_overlap = only_overlap_us.merge(only_overlap_other, left_on='city_ascii', right_on='city_ascii', suffixes=('_us', '_foreign'))
only_overlap.shape[0]

631

In [9]:
# Adding column for primary language spoken in the foreign country 
# (so that we can see pattern of how foreign culture influenced US regions)
country_languages = {
    'India': ['Hindi', 'English'],
    'Egypt': ['Arabic'],
    'Russia': ['Russian'],
    'France': ['French'],
    'United Kingdom': ['English'],
    'Peru': ['Spanish'],
    'Colombia': ['Spanish'],
    'Australia': ['English'],
    'Germany': ['German'],
    'Italy': ['Italian'],
    'Greece': ['Greek'],
    'Portugal': ['Portuguese'],
    'Syria': ['Arabic'],
    'Austria': ['German'],
    'Canada': ['English', 'French'],
    'Bolivia': ['Spanish', 'Quechua', 'Aymara'],
    'Poland': ['Polish'],
    'Uruguay': ['Spanish'],
    'Saudi Arabia': ['Arabic'],
    'Costa Rica': ['Spanish'],
    'Panama': ['Spanish'],
    'Switzerland': ['German', 'French', 'Italian', 'Romansh'],
    'Serbia': ['Serbian'],
    'Ireland': ['Irish', 'English'],
    'Liberia': ['English'],
    'Netherlands': ['Dutch'],
    'Jamaica': ['English'],
    'Sierra Leone': ['English'],
    'Latvia': ['Latvian'],
    'New Zealand': ['English', 'Maori'],
    'Guyana': ['English'],
    'Barbados': ['English'],
    'Seychelles': ['Seychellois Creole', 'English', 'French'],
    'San Marino': ['Italian'],
    'Sint Maarten': ['Dutch', 'English'],
    'Chile': ['Spanish'],
    'Argentina': ['Spanish'],
    'Iran': ['Persian'],
    'Paraguay': ['Spanish', 'Guarani'],
    'Mexico': ['Spanish'],
    'Puerto Rico': ['Spanish', 'English'],
    'Cuba': ['Spanish'],
    'Brazil': ['Portuguese'],
    'Philippines': ['Filipino', 'English'],
    'South Africa': ['Afrikaans', 'English', 'isiXhosa', 'isiZulu', 'Sesotho', 'Setswana', 'siSwati', 'Tshivenda', 'Xitsonga', 'Sepedi', 'isiNdebele'],
    'Spain': ['Spanish'],
    'El Salvador': ['Spanish'],
    'Trinidad And Tobago': ['English'],
    'Israel': ['Hebrew'],
    'Guatemala': ['Spanish'],
    'Venezuela': ['Spanish'],
    'Dominican Republic': ['Spanish'],
    'Romania': ['Romanian'],
    'Namibia': ['English'],
    'Honduras': ['Spanish'],
    'Nicaragua': ['Spanish'],
    'Belize': ['English'],
    'Malta': ['Maltese', 'English'],
    'Micronesia, Federated States Of': ['English'],
    'Sao Tome And Principe': ['Portuguese'],
    'Saint Helena, Ascension, And Tristan Da Cunha': ['English'],
    'Bermuda': ['English'],
    'Ecuador': ['Spanish'],
    'Isle Of Man': ['English', 'Manx Gaelic']
}

In [10]:
# Find most frequently spoken languages among those countries
all_languages = pd.Series([language for sublist in country_languages.values() for language in sublist]).value_counts()
most_frequent_lang = [l for l in all_languages.index if all_languages[l]>1]
most_frequent_lang

['English',
 'Spanish',
 'French',
 'Arabic',
 'German',
 'Italian',
 'Portuguese',
 'Dutch']

In [11]:
only_overlap.columns

Index(['city_us', 'city_ascii', 'lat_us', 'lng_us', 'country_us', 'iso2_us',
       'iso3_us', 'admin_name_us', 'capital_us', 'population_us', 'id_us',
       'city_foreign', 'lat_foreign', 'lng_foreign', 'country_foreign',
       'iso2_foreign', 'iso3_foreign', 'admin_name_foreign', 'capital_foreign',
       'population_foreign', 'id_foreign'],
      dtype='object')

In [12]:
# Add languages spoken in foreign countries to the dataframe
only_overlap['language'] = only_overlap['country_foreign'].map(country_languages)
only_overlap.head()

,city_us,city_ascii,lat_us,lng_us,country_us,iso2_us,iso3_us,admin_name_us,capital_us,population_us,...,lat_foreign,lng_foreign,country_foreign,iso2_foreign,iso3_foreign,admin_name_foreign,capital_foreign,population_foreign,id_foreign,language
0,Los Angeles,Los Angeles,34.1139,-118.4068,United States,US,USA,California,NaN,12815475.0,...,-37.4600,-72.36,Chile,CL,CHL,Biobío,minor,145239.0,1152810305,[Spanish]
1,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3603761.0,...,13.7000,-88.10,El Salvador,SV,SLV,Morazán,admin,16152.0,1222399599,[Spanish]
2,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3603761.0,...,-31.4300,-62.09,Argentina,AR,ARG,Córdoba,minor,59062.0,1032491643,[Spanish]
3,Portland,Portland,45.5371,-122.6500,United States,US,USA,Oregon,NaN,2052796.0,...,-38.3400,141.59,Australia,AU,AUS,Victoria,NaN,11808.0,1036439594,[English]
4,San Antonio,San Antonio,29.4658,-98.5254,United States,US,USA,Texas,NaN,2002530.0,...,-33.5995,-71.61,Chile,CL,CHL,Valparaíso,minor,104292.0,1152830501,[Spanish]


In [13]:
# Add separate column for the frequently spoken languages (so we can include it for direct comparison)
for lang in most_frequent_lang:
    only_overlap[lang] = only_overlap['language'].apply(lambda languages: lang in languages)

only_overlap.head()

,city_us,city_ascii,lat_us,lng_us,country_us,iso2_us,iso3_us,admin_name_us,capital_us,population_us,...,id_foreign,language,English,Spanish,French,Arabic,German,Italian,Portuguese,Dutch
0,Los Angeles,Los Angeles,34.1139,-118.4068,United States,US,USA,California,NaN,12815475.0,...,1152810305,[Spanish],False,True,False,False,False,False,False,False
1,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3603761.0,...,1222399599,[Spanish],False,True,False,False,False,False,False,False
2,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3603761.0,...,1032491643,[Spanish],False,True,False,False,False,False,False,False
3,Portland,Portland,45.5371,-122.6500,United States,US,USA,Oregon,NaN,2052796.0,...,1036439594,[English],True,False,False,False,False,False,False,False
4,San Antonio,San Antonio,29.4658,-98.5254,United States,US,USA,Texas,NaN,2002530.0,...,1152830501,[Spanish],False,True,False,False,False,False,False,False


# extra information from the website

In [14]:
cities = [
    "Athens", "Bath", "Berlin", "Bethlehem", "Brooklyn", "Brussels", "Cairo",
    "Cambridge", "Chinatown", "Damascus", "Durango", "Florence", "Frankfort",
    "Germantown", "Glasgow", "Gloucester", "Greenwich Village",
    "Hanover", "Harlem", "Havana", "Ithaca", "Lancaster", "Lebanon", "Lima",
    "Lisbon","Manchester","Melbourne","Milan","Montpelier","Moscow",
    "Naples","New Bern","Odessa","Olympia","Oxford","Paris","Plymouth","Portland",
    "Portsmouth","Rome","St.Petersburg","Strasburg","Syracuse","Toledo","Trinidad","Troy","Valparaiso",
    "Venice","Vienna","Waterbury","Zurich"]
missing = [i for i in cities if i not in set(only_overlap["city_us"])]

In [15]:
extra = pd.read_csv('extra.csv')
extra["language"] = extra['country_foreign'].map(country_languages)
##note: id_us are all null value, cannot find id_us online
## new language "Ukrainia" in the language column

In [16]:
extra = extra[extra['city_ascii']==extra['city_foreign']]

In [17]:
for lang in most_frequent_lang:
    extra[lang] = extra['language'].apply(lambda languages: lang in languages)

In [18]:
useful_cols = ['city_ascii', 'lat_us', 'lng_us', 'admin_name_us', 'country_foreign', 'language', 'English', 'Spanish', 'French', 'Arabic', 'German', 'Italian', 'Portuguese', 'Dutch']
df1 = only_overlap[useful_cols]
df2 = extra[useful_cols]
for i in range(df2.shape[0]):
    if df2.iloc[i]['city_ascii'] not in df1['city_ascii']:
        df1.loc[df1.shape[0]] = df2.iloc[i]
    else:
        if df2.iloc[i]['country_foreign'] not in df1[df1['city_ascii']==df2.iloc[i]['city_ascii']]['country_foreign']:
            df1.loc[df1.shape[0]] = df2.iloc[i]

/tmp/ipykernel_241/2847868487.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[df1.shape[0]] = df2.iloc[i]
/tmp/ipykernel_241/2847868487.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[df1.shape[0]] = df2.iloc[i]
/tmp/ipykernel_241/2847868487.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[df1.shape[0]] = df2.iloc[i]
/tmp/ipykernel_241/2847868487.py:6: SettingWithCopyWarning: 
A value is trying to be 

In [20]:
def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        else:
            flattened.append(item)
    return flattened
def trans_list(ser):
    in_list = flatten_list(ser.to_list())
    return np.unique(in_list)
a = df1.groupby(['city_ascii', 'lat_us', 'lng_us', 'admin_name_us']).language.apply(trans_list).reset_index()


In [21]:
# countries
func1 = lambda ser:ser.to_list()
b = df1.groupby(['city_ascii', 'lat_us', 'lng_us', 'admin_name_us']).country_foreign.apply(func1).reset_index()

In [22]:
# ohe language
func2 = lambda ser: any(ser)
c = df1.groupby(['city_ascii', 'lat_us', 'lng_us', 'admin_name_us']).agg({'English':func2, 'Spanish':func2, 'French':func2, 'Arabic':func2, 'German':func2, 'Italian':func2, 'Portuguese':func2, 'Dutch':func2}).reset_index()

In [23]:
final = a.merge(b[['country_foreign']], left_index = True, right_index = True)
final = final.merge(c[most_frequent_lang], left_index = True, right_index = True)
final

,city_ascii,lat_us,lng_us,admin_name_us,language,country_foreign,English,Spanish,French,Arabic,German,Italian,Portuguese,Dutch
0,Aberdeen,35.13510,-79.43190,North Carolina,[English],[United Kingdom],True,False,False,False,False,False,False,False
1,Aberdeen,39.51460,-76.17300,Maryland,[English],[United Kingdom],True,False,False,False,False,False,False,False
2,Aberdeen,40.41650,-74.22490,New Jersey,[English],[United Kingdom],True,False,False,False,False,False,False,False
3,Aberdeen,45.46460,-98.46800,South Dakota,[English],[United Kingdom],True,False,False,False,False,False,False,False
4,Aberdeen,46.97570,-123.80950,Washington,[English],[United Kingdom],True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,York,34.99670,-81.23400,South Carolina,[English],[United Kingdom],True,False,False,False,False,False,False,False
479,York,39.96510,-76.73150,Pennsylvania,[English],[United Kingdom],True,False,False,False,False,False,False,False
480,York,40.86980,-97.59430,Nebraska,[English],[United Kingdom],True,False,False,False,False,False,False,False
481,York,43.18600,-70.66610,Maine,[English],[United Kingdom],True,False,False,False,False,False,False,False


In [44]:
final.to_csv('output.csv', index=False)